# Getting Started with Segmentations

## Before you start!

- This [notebook](getting-started-with-segmentations.ipynb) assumes that shapeworks conda environment has been activated using `conda activate shapeworks` on the terminal.
- See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb) to learn how to set up your environment to start using shapeworks library. Please note, the prerequisite steps will use the same code to setup the environment for this notebook and import `shapeworks` library.


## In this notebook, you will learn:

1. How to define your dataset location and explore what is available in it
2. How to load a single segmentation 
3. How to convert `shapeworks` image to `vtk` image for visualization
4. How to visualize a segmentation using `itkwidgets`
5. How to visualize a segmentation using `pyvista`
6. How to visualize two segmentations side-by-side using `pyvista`
7. How to visualize two segmentations in the same rendering window using `pyvista` 

We will also define modular/generic helper functions as we walk through these items to reuse functionalities without duplicating code.


## Prerequisites

- Setting up `shapeworks` environment. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb). To avoid code clutter, the `setup_shapeworks_env` function can found in `Examples/Python/setupenv.py` module. 
- Importing `shapeworks` library. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb).

## Note about `shapeworks` APIs

shapeworks functions are inplace, i.e., `<swObject>.<function>()` applies that function to the `swObject` data. To keep the original data unchanged, you have first to copy it to another variable before applying the function.

## Notebook keyboard shortcuts

- `Esc + H`: displays a complete list of keyboard shortcuts
- `Esc + A`: insert new cell above the current cell
- `Esc + B`: insert new cell below the current cell
- `Esc + D + D`: delete current cell
- `Esc + Z`: undo
- `Shift + enter`: run current cell and move to next
- To show a function's argument list (i.e., signature), use `(` then `shift-tab`
- Use `shift-tab-tab` to show more help for a function
- To show the help of a function, use `help(function)` or `function?`
- To show all functions supported by an object, use `dot-tab` after the variable name

## Prerequisites

### Setting up `shapeworks` environment 

Here, we will append both your `PYTHONPATH` and your system `PATH` to setup shapeworks environment for this notebook. See [Setting Up ShapeWorks Environment](setting-up-shapeworks-environment.ipynb) for more details.

In this notebook, we assume the following.

- This notebook is located in `Examples/Python/notebooks/tutorials`
- You have built shapeworks from source in `build` directory within the shapeworks code directory
- You have built shapeworks dependencies (using `build_dependencies.sh`) in the same parent directory of shapeworks code

**Note:** If you run from a ShapeWorks installation, you don't need to set the dependencies path and the `shapeworks_bin_dir` would be set as `../../../../bin`.

In [ ]:
# import relevant libraries 
import sys 

# add parent-parent directory (where setupenv.py is) to python path
sys.path.insert(0,'../..')

# importing setupenv from Examples/Python
import setupenv

# indicate the bin directories for shapeworks and its dependencies
shapeworks_bin_dir   = "../../../../build/bin"
dependencies_bin_dir = "../../../../../shapeworks-dependencies/bin"

# set up shapeworks environment
setupenv.setup_shapeworks_env(shapeworks_bin_dir,  
                              dependencies_bin_dir, 
                              verbose = False)

### Importing `shapeworks` library

In [ ]:
# let's import shapeworks library to test whether shapeworks is now set
try:
    import shapeworks as sw
except ImportError:
    print('ERROR: shapeworks library failed to import')
else:
    print('SUCCESS: shapeworks library is successfully imported!!!')

## 1. Defining and exploring your dataset

### Defining dataset location

You can download exemplar datasets from [ShapeWorks data portal](http://cibc1.sci.utah.edu:8080) after you login. For new users, you can [register](http://cibc1.sci.utah.edu:8080/#?dialog=register) an account for free. Please do not use an important password.

After you login, click `Collections` on the left panel and then `use-case-data-v2`. Select the dataset you would like to download by clicking on the checkbox on the left of the dataset name. See the video below.

**This notebook assumes that you have downloaded `ellipsoid-v2` in `Examples/Python/Data`.** Feel free to use your own dataset. 


<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/portal_data_download.mp4" autoplay muted loop controls style="width:100%"></p>


In [ ]:
import os # for paths and mkdir

# dataset name is the folder name for your dataset
datasetName  = 'ellipsoid-v2'

# path to the dataset where we can find shape data 
# here we assume shape data are given as binary segmentations
shapeDir      = '../../Data/' + datasetName + '/segmentations/'

print('Dataset Name:     ' + datasetName)
print('Shape Directory:  ' + shapeDir)

### What is available in the dataset?

First let's see how many shapes we have in the dataset.

**File formats:** For binary segmentations, all [itk-supported image formats](https://insightsoftwareconsortium.github.io/itk-js/docs/image_formats.html) can be used.

In [ ]:
import glob # for paths and file-directory search

# file extension for the shape data
shapeExtention = '.nrrd'

# let's get a list of files for available segmentations in this dataset
# * here is a wild character used to retrieve all filenames 
# in the shape directory with the file extensnion
shapeFilenames = sorted(glob.glob(shapeDir + '*' + shapeExtention)) 

print ('Number of shapes: ' + str(len(shapeFilenames)))
print('Shape files found:')
for shapeFilename in shapeFilenames:
    print('\t' + shapeFilename)

## 2. Loading a single segmentation

We will select one segmentation to explore for now. We will then use shapeworks Image class to load this segmentation and print out its header information that includes image dimensions (rows, columns and slices), physical origin, physical size, and voxel spacing (in physical units).

In [ ]:
# select a shape by setting the shape index (in the filenames list)
shapeIdx       = 1

# the filename for the selected shape
shapeFilename  = shapeFilenames[shapeIdx]

# since segmentation is just an image data, we will use shapeworks Image class to load it
print('Loading: ' + shapeFilename)
shapeSeg = sw.Image(shapeFilename)

# let's print out header information of this segmentation 
print('Header information: ')
print(shapeSeg)

## 3. Converting `shapeworks` image to `vtk` image for visualization

We can use python libraries such as [`itkwidgets`](https://github.com/InsightSoftwareConsortium/itkwidgets) and [`pyvista`](https://docs.pyvista.org/index.html) for interactive 3D visualization. These libraries support, among others, `vtk` data structures for images and meshes. Hence, to visualize our `shapeworks` image, we need first to convert it to a `vtk` data structure.

This conversion can be performed by first extracting a numpy array from the `shapeworks` image, then constructing a `vtk` image from this array. `pyvista`'s `wrap` function makes this easy.

### Extracting `numpy` array from `shapeworks` image

In [ ]:
# first, let's extract the numpy array of the shapeworks Image object
shapeSeg_array = shapeSeg.toArray()

# then compare their size
print('shapeworks image size: ')
print(shapeSeg.dims())

print('numpy array size: ')
print(shapeSeg_array.shape) 

In [ ]:
# notice that the numpy array needs to be permuted to match the shapeworks image dimensions
import numpy as np

shapeSeg_array = np.transpose(shapeSeg_array,(2,1,0))

print('shapeworks image size: ')
print(shapeSeg.dims())

print('numpy array size: ')
print(shapeSeg_array.shape) 

### Constructing a `vtk` image from a `numpy` array

In [ ]:
# construct a vtk image from the extracted numpy array using pyvista wrap function
import pyvista as pv

# converting a numpy array to a vtk image using pyvista's wrap function
shapeSeg_vtk = pv.wrap(shapeSeg_array)

# print header info of both images, please note that the wrap function
# only wraps the numpy array, which is oblivous to image spacing and origin
print('shapeworks image header information: ')
print(shapeSeg)

print('\nvtk image header information: ')
print(shapeSeg_vtk) 

### Defining a helper function

As converting between `shapeworks` Image object and `vtk` image is a step that we will need frequently, let's add a helper function for this purpose.

In [ ]:
# a helper function that converts shapeworks Image object to vtk image
def sw2vtkImage(swImg, verbose = False):
            
    # get the numpy array of the shapeworks image
    array  = swImg.toArray()
    
    # the numpy array needs to be permuted to match the shapeworks image dimensions
    array = np.transpose(array,(2,1,0))
    
    # converting a numpy array to a vtk image using pyvista's wrap function
    vtkImg = pv.wrap(array)
    
    if verbose:
        print('shapeworks image header information: ')
        print(swImg)

        print('\nvtk image header information: ')
        print(vtkImg) 
    
    return vtkImg

## 4. Visualizing segmentation using `itkwidgets`

[`itkwidgets`](https://github.com/InsightSoftwareConsortium/itkwidgets) is a python library that supports interactive Jupyter widgets to visualize images, point sets, and meshes. 

`itkwidgets` supports `itk`, `vtk`, and `pyvista` data structures. Hence, to visualize a `shapeworks` image, we need first to convert it to a `vtk` image using the `sw2vtkImage` helper function that we just defined.


In [ ]:
# convert shapeworks images to a vtk image
shapeSeg_vtk = sw2vtkImage(shapeSeg, verbose = True)

Now we have a `vtk` image, we can simply visualize and interact with it using the `view` function of `itkwidgets`.

In [ ]:
# importing itkwidgets
import itkwidgets as itkw

# visualize - this is a volume rendering of the shape segmentation
itkw.view(shapeSeg_vtk)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-seg-vol.mp4" autoplay muted loop controls style="width:100%"></p>

Let's visualize the segmentation with orthogonal image planes to better understand the spatial relation of the segmentation and image boundaries.

In [ ]:
# visualize - this time we will also visualize orthogonal image planes
itkw.view(  image          = shapeSeg_vtk, # for orthoginal image plane
            slicing_planes = True, 
            rotate         = True, # enable auto rotation
            axes           = True)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-seg-vol-slices.mp4" autoplay muted loop controls style="width:100%"></p>

To also visualize the segmentation as a label map (binary image), we need add it as a label image and use the label map blend.

In [ ]:
# to visualize label map - use label map blend 
itkw.view( image          = shapeSeg_vtk, # for orthoginal image plane
           label_image    = shapeSeg_vtk,  # for volume rendering segmentation
           slicing_planes = True, 
           axes           = True,
           rotate         = True, # enable auto rotation
           interpolation  = True)

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-seg-vol-slices-label.mp4" autoplay muted loop controls style="width:100%"></p>

## 5. Visualizing segmentation using `pyvista`

[`pyvista`](https://docs.pyvista.org/index.html) is a python library for 3D visualization and analysis. It is built on top of `vtk` and brings a paraview-like visualizations to notebooks. It also supports multiple rendering windows that can be linked. This feature is very useful when visualizing multiple samples from your dataset side-by-side and making them share the same camera view.

`pyvista` supports `vtk` data structures. Hence, to visualize a `shapeworks` image, we need first to convert it to a `vtk` image.

In [ ]:
# volume rendering the segmentation... notice the static view!
shapeSeg_vtk.plot(volume = True,   # volume render
                  shade  = True)   # enable shading

In [ ]:
# to have an interactive visualization, 
# we need enable use_ipyvtk for this plot 
# click r to reset the view after zooming
shapeSeg_vtk.plot(volume     = True,     # volume render
                  shade      = True,     # enable shading
                  use_ipyvtk = True) # enable interactive plots

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-seg-pv-vol.mp4" autoplay muted loop controls style="width:100%"></p>

In [ ]:
# Or, we can enable use_ipyvtk by default for interactive plots
pv.rcParams['use_ipyvtk'] = True 

# click r to reset the view after zooming
shapeSeg_vtk.plot(volume = True,  # volume render
                  shade  = True)   # enable shading

## 6. Visualizing two segmentations side-by-side using `pyvista`

When exploring datasets and results of different grooming (data preprocessing) steps, it is important to simultaneously visualize multiple shape samples. Here, we will learn how to visualize two segmentations side-by-side and link their views using `pyvista`. This linking is useful to make all rendering windows share the same camera view.  

### Loading the second segmentation and convert it to `vtk` image

First, let's select another segmentation and load it.

In [ ]:
# select a shape by setting the shape index (in the filenames list)
shapeIdx2       = 2

# the filename for the selected shape
shapeFilename2  = shapeFilenames[shapeIdx2]

# since segmentation is just an image data, we will use shapeworks Image class to load it
print('Loading: ' + shapeFilename2)
shapeSeg2 = sw.Image(shapeFilename2)

# let's print out header information of this segmentation 
print('Header information: ')
print(shapeSeg2)

Then, let's convert this `shapeworks` image to a `vtk` image for visualization.

In [ ]:
# sw to vtk
shapeSeg2_vtk = sw2vtkImage(shapeSeg2)

### Defining `pyvista` plotter

Next, we will define a `pyvista` plotter to render multiple windows, each with a single segmentation. The multiple rendering windows will be visualized as a grid of plots. Since, we have only two segmentations, the grid size will be one row and two columns.

The plotter also enable use to specify a color map.

In [ ]:
# define grid size for two segmentations
grid_rows  = 1
grid_cols  = 2

# define parameters that controls the plotter
is_interactive = True  # to enable interactive plots
show_borders   = True  # show borders for each rendering window
shade_volumes  = True  # use shading when performing volume rendering
color_map      = "coolwarm" # color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
show_axes      = True  # show a vtk axes widget for each rendering window
show_bounds    = False # show volume bounding box
show_all_edges = True  # add an unlabeled and unticked box at the boundaries of plot. 
font_size      = 10    # text font size for windows
link_views     = True  # link all rendering windows so that they share same camera and axes boundaries

# define the plotter
plotter = pv.Plotter(shape    = (grid_rows, grid_cols),
                     notebook = is_interactive, 
                     border   = show_borders) 

### Adding segmentations to the plotter and start rendering

Let's add the two segmentations to the plotter and start the viz fun!

In [ ]:
# add the first segmentation
plotter.subplot(0, 0)
plotter.add_volume(shapeSeg_vtk, shade = shade_volumes, cmap = color_map)

if show_axes:
    plotter.show_axes()
    
if show_bounds:
    plotter.show_bounds(all_edges = show_all_edges)

# add a text to this subplot to indicate which segmentation is being visualized
segFilename = shapeFilenames[shapeIdx].split('/')[-1] 
shapeName   = segFilename[:-len(shapeExtention)]
plotter.add_text(shapeName, font_size = font_size)

# now, add the second segmentation, 
# note that we repeat the same exact code but with a different segmentation 
# ---> perfect scenario to define a helper function 
# to reuse this code without having to duplicate the code    
plotter.subplot(0, 1)
plotter.add_volume(shapeSeg2_vtk, shade = shade_volumes, cmap = color_map)

if show_axes:
    plotter.show_axes()
    
if show_bounds:
    plotter.show_bounds(all_edges = show_all_edges)

# add a text to this subplot to indicate which segmentation is being visualized
segFilename2 = shapeFilenames[shapeIdx2].split('/')[-1] 
shapeName2   = segFilename2[:-len(shapeExtention)]
plotter.add_text(shapeName2, font_size = font_size)

# link views
if link_views:
    plotter.link_views()  

# now, time to render our segmentations
plotter.show() #use_ipyvtk=True is already enabled by default using pv.rcParams

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-seg-pv-2vols.mp4" autoplay muted loop controls style="width:100%"></p>

### Defining a helper function

Let's define a helper function that adds a segmentation to a `pyvista` plotter.

In [ ]:
def add_volume_to_plotter( pvPlotter,      # pyvista plotter
                           vtkImg,         # vtk image to be added
                           rowIdx, colIdx, # subplot row and column index
                           title = None,   # text to be added to the subplot, use None to not show text 
                           shade_volumes  = True,  # use shading when performing volume rendering
                           color_map      = "coolwarm", # color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
                           show_axes      = True,  # show a vtk axes widget for each rendering window
                           show_bounds    = False, # show volume bounding box
                           show_all_edges = True,  # add an unlabeled and unticked box at the boundaries of plot. 
                           font_size      = 10     # text font size for windows
                         ):
    
    # which subplot to add the volume to
    pvPlotter.subplot(rowIdx, colIdx)
    
    # add the volume
    pvPlotter.add_volume(vtkImg, 
                         shade   = shade_volumes, 
                         cmap    = color_map)

    if show_axes:
        pvPlotter.show_axes()

    if show_bounds:
        pvPlotter.show_bounds(all_edges = show_all_edges)

    # add a text to this subplot to indicate which volume is being visualized
    if title is not None:
        pvPlotter.add_text(title, font_size = font_size)

Let's test the helper functions by adding both segmentations then render.

In [ ]:
# define the plotter
plotter = pv.Plotter(shape    = (grid_rows, grid_cols),
                     notebook = is_interactive, 
                     border   = show_borders) 

# add the first segmentation
add_volume_to_plotter( plotter, shapeSeg_vtk,   
                       rowIdx = 0, colIdx = 0, 
                       title          = shapeName,
                       shade_volumes  = shade_volumes, 
                       color_map      = color_map,
                       show_axes      = show_axes, 
                       show_bounds    = show_bounds, 
                       show_all_edges = show_all_edges, 
                       font_size      = font_size)

# add the second segmentation - note that we could define a loop to avoid code repetition
add_volume_to_plotter( plotter, shapeSeg2_vtk,   
                       rowIdx = 0, colIdx = 1, 
                       title          = shapeName2,
                       shade_volumes  = shade_volumes, 
                       color_map      = color_map,
                       show_axes      = show_axes, 
                       show_bounds    = show_bounds, 
                       show_all_edges = show_all_edges, 
                       font_size      = font_size)

# link views
if link_views:
    plotter.link_views()  

# now, time to render our segmentations
plotter.show() #use_ipyvtk=True is already enabled by default using pv.rcParams

## 7. Visualizing two segmentations in the same rendering window

This type of visualization is useful when exploring differences between more than one segmentations, e.g., when inspecting the impact of a grooming/preprocessing step or the spatial relation of multiple samples. This is also useful if your shape data contains multiple domains (or compartments) such as anatomical joints.

Note that, since we have a single rendering window (view), linking views is not necessary. But, if this multi-surface visualization is used in conjuction with multiple rendering windows, linking views should be considered. 

###  Using `pyvista` 

Note that, since we have a single rendering window (view), linking views is not necessary. But, if this multi-surface visualization is used in conjuction with multiple rendering windows, linking views should be considered. 

In [ ]:
# since we want to visualize the two segmentations in the same rendering window
# we define grid size for two meshes as 1 x 1 grid

grid_rows  = 1
grid_cols  = 1

# define parameters that controls the plotter
is_interactive = True  # to enable interactive plots
show_borders   = True  # show borders for each rendering window
shade_volumes  = True  # use shading when performing volume rendering
show_axes      = True  # show a vtk axes widget for each rendering window
show_bounds    = False # show volume bounding box
show_all_edges = True  # add an unlabeled and unticked box at the boundaries of plot. 
font_size      = 10    # text font size for windows
link_views     = True  # link all rendering windows so that they share same camera and axes boundaries

# let'd define a different color map for each segmentation
# color map for volume rendering, e.g., 'bone', 'coolwarm', 'cool', 'viridis', 'magma'
color_map1     = "bone" 
color_map2     = "coolwarm" 

one_title      = "%s (%s), %s (%s)" % (shapeName, color_map1, shapeName2, color_map2)

# define the plotter
plotter = pv.Plotter(shape    = (grid_rows, grid_cols),
                     notebook = is_interactive, 
                     border   = show_borders) 

# add the first segmentation
add_volume_to_plotter( plotter, shapeSeg_vtk,   
                       rowIdx = 0, colIdx = 0, 
                       title          = None,
                       shade_volumes  = shade_volumes, 
                       color_map      = color_map1,
                       show_axes      = show_axes, 
                       show_bounds    = show_bounds, 
                       show_all_edges = show_all_edges, 
                       font_size      = font_size)

# add the second segmentation - note that we could define a loop to avoid code repetition
add_volume_to_plotter( plotter, shapeSeg2_vtk,   
                       rowIdx = 0, colIdx = 0, 
                       title          = one_title,
                       shade_volumes  = shade_volumes, 
                       color_map      = color_map2,
                       show_axes      = show_axes, 
                       show_bounds    = show_bounds, 
                       show_all_edges = show_all_edges, 
                       font_size      = font_size)

# now, time to render our segmentations
plotter.show() #use_ipyvtk=True is already enabled by default using pv.rcParams

<p><video src="https://sci.utah.edu/~shapeworks/doc-resources/mp4s/nb-seg-pv-2vols-same-win.mp4" autoplay muted loop controls style="width:100%"></p>